In [1]:
import math
import casadi as ca
import numpy as np

# ----------------------------------------------------------------------
# 1.  tiny‑angle Maclaurin (degree 6 ⇒ O(θ⁸) remainder)
# ----------------------------------------------------------------------
def _cm_maclaurin(theta, m):
    """
    Return c_m(θ) by summing terms up to θ⁶.
    Works with SX because the loop is unrolled at Python level.
    """
    theta2 = theta * theta
    term = 1.0 / math.factorial(m + 2)   # k = 0
    acc  = term

    # k = 1 … 3   (adds θ², θ⁴, θ⁶ terms)
    for k in range(1, 4):
        term *= -theta2 / ((2*k + m + 1) * (2*k + m + 2))
        acc  += term
    return acc


# ----------------------------------------------------------------------
# 2.  closed‑form remainder (unchanged from before)
# ----------------------------------------------------------------------
def _taylor_trunc(theta, p: int, odd: bool):
    r = 1 if odd else 0
    poly = ((-1)**p) / math.factorial(2*p + r)
    for j in range(p-1, -1, -1):
        poly = poly * theta**2 + ((-1)**j) / math.factorial(2*j + r)
    return poly * (theta if odd else 1)

def _c_m_closed(theta, m: int):
    p, r = divmod(m, 2)
    if r == 0:
        rem = ca.cos(theta) - _taylor_trunc(theta, p, odd=False)
    else:
        rem = ca.sin(theta) - _taylor_trunc(theta, p, odd=True)
    return ((-1)**(p + 1) / theta**(m + 2)) * rem


def c_m_safe(theta, m, eps=1e-4):
    """
    Symbolic run‑time switch:
       |θ| < eps → Maclaurin   (high relative accuracy)
       else       → closed form
    """
    return ca.if_else(ca.fabs(theta) < eps,
                      _cm_maclaurin(theta, m),
                      _c_m_closed(theta, m))


def cm_table(theta, l, eps=1e-4):
    """Vector [c₀ … c_{l+1}]ᵀ using the safe formula."""
    return ca.vcat([c_m_safe(theta, m, eps) for m in range(l + 2)])


# -----------------------------------------------------------------------------
# 2.  factory for  P(Ω, A)  with truncation index nil_deg   (SX symbols only)
# -----------------------------------------------------------------------------
def P(omega, A, B, nil_deg):
    """
    Parameters
    ----------
    B : nil-potent square matrix
    nil_deg: nilpotency degree of B
    Returns
    -------
    ca.Function   P(theta, Omega, A, B) -> n×n SX matrix
    """


    # ---------- symbolic inputs ----------
    Omega = ca.skew(omega)
    theta = ca.norm_2(omega)
    n = B.shape[0]

    # ---------- scalar coefficient tables ----------
    alpha = [1.0 / math.factorial(m + 1) for m in range(nil_deg)]   # 1/(m+1)!
    cm = cm_table(theta, nil_deg)                                  # [c₀ … c_{l+1}]

    # ---------- shared‑power loop ----------
    S0 = ca.SX.zeros(n, n)
    S1 = ca.SX.zeros(n, n)
    S2 = ca.SX.zeros(n, n)
    T  = ca.SX.eye(n) # B⁰

    for m in range(nil_deg):
        S0 += alpha[m] * T
        S1 += cm[m] * T
        S2 += cm[m+1] * T
        T = T @ B

    OmegaA = Omega@A

    # ---------- assemble P ----------
    return A @ S0 + OmegaA @ S1 + Omega @ OmegaA @ S2

# -----------------------------------------------------------------------------
# 3.  quick demo
# -----------------------------------------------------------------------------
if __name__ == '__main__':
    omega = np.array([0.1, 0.2, 0.3], float)
    B     = np.array([[0,1,0],[0,0,0],[0,0,0]], float)
    A     = np.random.rand(3, B.shape[0])
    P0 = P(omega, A, B, 1)
    print("P =", P0)

P = 
[[0.365505, 0.26398, 0.141989], 
 [0.337595, 0.207798, 0.190704], 
 [0.0173872, 0.702802, 0.375819]]


In [ ]:
from cyecca import lie

def exp_mixed_old(
    self,
    X0: lie.group_se23.SE23LieGroupElement,
    l: lie.group_se23.SE23LieAlgebraElement,
    r: lie.group_se23.SE23LieAlgebraElement,
    B: ca.SX,
) -> lie.group_se23.SE23LieGroupElement:
    n = B.shape[0]
    P0 = ca.horzcat(X0.v.param, X0.p.param)
    Pl = self.calculate_N(l, B)
    Pr = self.calculate_N(r, -B)
    R0 = X0.R
    Rl = (l).Omega.exp(self.SO3)
    Rr = (r).Omega.exp(self.SO3)
    Rr0 = Rr * R0
    R1 = Rr0 * Rl

    I = ca.SX.eye(n)
    P1 = Rr0.to_Matrix() @ Pl + (Rr.to_Matrix() @ P0 + Pr) @ (I + B)
    return self.elem(ca.vertcat(P1[:, 1], P1[:, 0], R1.param))

def exp_mixed_new(
    self,
    X0: lie.group_se23.SE23LieGroupElement,
    l: lie.group_se23.SE23LieAlgebraElement,
    r: lie.group_se23.SE23LieAlgebraElement,
    B: ca.SX,
    nil_deg: int,
) -> lie.group_se23.SE23LieGroupElement:
    n = B.shape[0]
    P0 = ca.horzcat(X0.v.param, X0.p.param)
    
    Al = ca.horzcat(l.param[3:6], l.param[0:3])
    Ar = ca.horzcat(r.param[3:6], r.param[0:3])

    Pl = P(l.param[6:9], Al, B, nil_deg)
    Pr = P(r.param[6:9], Ar, -B, nil_deg)

    R0 = X0.R
    Rl = (l).Omega.exp(self.SO3)
    Rr = (r).Omega.exp(self.SO3)
    Rr0 = Rr * R0
    R1 = Rr0 * Rl

    I = ca.SX.eye(n)
    P1 = Rr0.to_Matrix() @ Pl + (Rr.to_Matrix() @ P0 + Pr) @ (I + B)

    return self.elem(ca.vertcat(P1[:, 1], P1[:, 0], R1.param))

def find_flops(op_dict):
    flops = 0
    for k, v in op_dict.items():
        if k not in ["OP_PARAMETER", "OP_CONST"]:
            flops += v
    return flops

In [24]:
dt = ca.SX.sym("dt")
X0 = lie.SE23Quat.elem(ca.SX.sym("X0", 10))
a_b = ca.SX.sym("a_b", 3)
g = ca.SX.sym("g")
omega_b = ca.SX.sym("omega_b", 3)
l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))


X1 = exp_mixed_new(lie.SE3Quat, X0, l * dt, r * dt, B * dt, 2)
X1


AssertionError: 

In [ ]:
from cyecca import lie
from cyecca.symbolic import taylor_series_near_zero

def derive_strapdown_ins_propagation_old():
    dt = ca.SX.sym("dt")
    X0 = lie.SE23Quat.elem(ca.SX.sym("X0", 10))
    a_b = ca.SX.sym("a_b", 3)
    g = ca.SX.sym("g")
    omega_b = ca.SX.sym("omega_b", 3)
    l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
    r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
    B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))
    X1 = exp_mixed_old(lie.SE23Quat, X0, l * dt, r * dt, B * dt)
    return ca.Function(
        "strapdown_ins_propagate",
        [X0.param, a_b, omega_b, g, dt],
        [X1.param],
        ["x0", "a_b", "omega_b", "g", "dt"],
        ["x1"],
    )

def derive_strapdown_ins_propagation_new():
    dt = ca.SX.sym("dt")
    X0 = lie.SE23Quat.elem(ca.SX.sym("X0", 10))
    a_b = ca.SX.sym("a_b", 3)
    g = ca.SX.sym("g")
    omega_b = ca.SX.sym("omega_b", 3)
    l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
    r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
    B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))
    X1 = exp_mixed_new(lie.SE23Quat, X0, l * dt, r * dt, B * dt, 2)
    return ca.Function(
        "strapdown_ins_propagate",
        [X0.param, a_b, omega_b, g, dt],
        [X1.param],
        ["x0", "a_b", "omega_b", "g", "dt"],
        ["x1"],
    )

def derive_strapdown_ins_propagation_barfoot():
    dt = ca.SX.sym("dt")
    X0 = lie.SE23Quat.elem(ca.SX.sym("X0", 10))
    a_b = ca.SX.sym("a_b", 3)
    g = ca.SX.sym("g")
    omega_b = ca.SX.sym("omega_b", 3)
    theta = ca.norm_2(omega_b)

    R = X0.R * lie.so3.elem(omega_b * dt).exp(lie.SO3Quat)



    c1 = ca.if_else(ca.fabs(theta) < 1e-4, 2*(1 - ca.cos(theta))/theta**2, 0)
    c2 = ca.if_else(ca.fabs(theta) < 1e-4, 1 - 2*(1 - ca.cos(theta))/theta**2, 0)
    c3 = ca.if_else(ca.fabs(theta) < 1e-4, 2*(theta - ca.sin(theta))/theta**2, 0)

    omega_b_dt = omega_b * dt

    a = omega_b_dt/theta

    N = c1*ca.eye(3) + c2*a.T@a + c2*ca.skew(a)


    N = 2*(1 - ca.cos(theta))/theta**2 + (1 - 2*(1 - ca.cos(theta)))/theta**2 + 


    l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
    r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
    B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))
    X1 = exp_mixed_new(lie.SE23Quat, X0, l * dt, r * dt, B * dt, 2)
    return ca.Function(
        "strapdown_ins_propagate",
        [X0.param, a_b, omega_b, g, dt],
        [X1.param],
        ["x0", "a_b", "omega_b", "g", "dt"],
        ["x1"],
    )

SyntaxError: invalid syntax (3988857950.py, line 53)

In [31]:
from cyecca.util import count_ops
omega = ca.SX.sym('omega', 3)
A = ca.SX.sym('A', 3, B.shape[0])

sins_old = derive_strapdown_ins_propagation_old()
sins_sym_old = sins_old(
    ca.SX.sym("x0", 10),
    ca.SX.sym("a_b", 3),
    ca.SX.sym("omega_b", 3),
    ca.SX.sym("g", 1),
    ca.SX.sym("dt", 1),
)

sins_new = derive_strapdown_ins_propagation_new()
sins_sym_new = sins_new(
    ca.SX.sym("x0", 10),
    ca.SX.sym("a_b", 3),
    ca.SX.sym("omega_b", 3),
    ca.SX.sym("g", 1),
    ca.SX.sym("dt", 1),
)

op_dict_old = count_ops(sins_sym_old)
flops_old = find_flops(op_dict_old)
print("Old flops:", flops_old)

op_dict_new = count_ops(sins_sym_new)
flops_new = find_flops(op_dict_new)
print("New flops:", flops_new)


building dependency graph...done
Old flops: 342
building dependency graph...done
New flops: 291


In [43]:
x0 = np.random.randn(10)
x0[6:] = [1, 0, 0, 0]
a_b = np.random.randn(3)
omega_b = np.random.randn(3)
g = 9.8
dt = 10
sins_old(x0, a_b, omega_b, g, dt) - sins_new(x0, a_b, omega_b, g, dt)

DM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])